<a href="https://colab.research.google.com/github/binoyskaria/google_collab_projects/blob/main/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from collections import Counter

# Load the dataset
df = pd.read_csv("/content/Q1Data.csv")
df = df.sample(frac=0.6, random_state=42)

# Assuming the MFCC features are in columns 1 to 21 and the target variable (Genus) is in the last column
X = df.iloc[:, 1:-3].values
y = df['Genus'].values

# Convert categorical labels to numeric labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into train and test sets (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Part 1: Scratch Decision Tree
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
# Part 1a: Train a Decision Tree Classifier
print("Decision Tree Classifier trained.")

# Part 1b: Plot the test accuracy by pruning the tree to a depth ranging from 1 to 15
depths = list(range(1, 16))
test_accuracies_1 = []

for depth in depths:
    # Prune the tree to the specified depth
    tree_classifier = DecisionTree(min_samples_split=5, max_depth=depth, n_features=21)
    tree_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = tree_classifier.predict(X_test)

    # Calculate accuracy
    accuracy = np.mean(predictions == y_test)
    test_accuracies_1.append(accuracy)

# Plotting
# plt.plot(depths, test_accuracies_1, marker='o')
# plt.title('Test Accuracy vs. Tree Depth')
# plt.xlabel('Tree Depth')
# plt.ylabel('Test Accuracy')
# plt.grid(True)

# Part 1c: Observations
# Your observations can be made based on the plotted graph. Look for the point where the test accuracy is optimal.
# This is usually the point before overfitting occurs. Observe how the test accuracy changes with different depths.

#Part 2: Scratch Knn

def euclidean_distance(x1, x2):
    distance = np.sqrt(np.sum((x1 - x2) ** 2))
    return distance

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions

    def _predict(self, x):
        # compute the distance
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # get the closest k
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # majority vote
        most_common = Counter(k_nearest_labels).most_common()
        return most_common[0][0]

# Part 2a: Train a kNN Classifier

print("kNN Classifier trained.")

# Part 2b: Plot test accuracy for k varying from 1 to 1000
max_k = 50
k_values = list(range(1, max_k + 1))
test_accuracies_2 = []

for k in k_values:
    knn_classifier = KNN(k)
    knn_classifier.fit(X_train, y_train)
    predictions = knn_classifier.predict(X_test)
    accuracy = np.mean(predictions == y_test)
    test_accuracies_2.append(accuracy)



# Plotting
# plt.plot(k_values, test_accuracies_2, marker='o')
# plt.title('Test Accuracy vs. k for kNN')
# plt.xlabel('k (Number of Neighbors)')
# plt.ylabel('Test Accuracy')
# plt.grid(True)

# Part 2c: Observations
# Your observations can be made based on the plotted graph. Look for the point where the test accuracy is optimal.
# This is usually the point before overfitting occurs. Observe how the test accuracy changes with different k values.






# Part 3a: Decision Trees using scikit-learn
depths = list(range(1, 16))
dt_test_accuracies = []

for depth in depths:
    dt_classifier = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt_classifier.fit(X_train, y_train)
    dt_predictions = dt_classifier.predict(X_test)
    dt_accuracy = accuracy_score(y_test, dt_predictions)
    dt_test_accuracies.append(dt_accuracy)

# Part 3b: kNN using scikit-learn
max_k = 1000
k_values = list(range(1, max_k + 1))
knn_test_accuracies = []

for k in k_values:
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    knn_classifier.fit(X_train, y_train)
    knn_predictions = knn_classifier.predict(X_test)
    knn_accuracy = accuracy_score(y_test, knn_predictions)
    knn_test_accuracies.append(knn_accuracy)

# Part 3c: Plotting
plt.figure(figsize=(12, 8))

# Plot for Decision Trees
plt.subplot(2, 2, 1)
plt.plot(depths, dt_test_accuracies, marker='o')
plt.title('Decision Tree Test Accuracy vs. Depth')
plt.xlabel('Tree Depth')
plt.ylabel('Test Accuracy')
plt.grid(True)

# Plot for kNN
plt.subplot(2, 2, 2)
plt.plot(k_values, knn_test_accuracies, marker='o')
plt.title('kNN Test Accuracy vs. k')
plt.xlabel('k (Number of Neighbors)')
plt.ylabel('Test Accuracy')
plt.grid(True)

# # Part 3d: Comparing models built from Scratch and scikit-learn
# # Plot for Decision Trees (built from Scratch)
plt.subplot(2, 2, 3)
plt.plot(depths, test_accuracies_1, marker='o', label='Scratch')
plt.title('Decision Tree (Scratch) Test Accuracy vs. Depth')
plt.xlabel('Tree Depth')
plt.ylabel('Test Accuracy')
plt.grid(True)

# Plot for kNN (built from Scratch)
plt.subplot(2, 2, 4)
plt.plot(k_values, test_accuracies_2, marker='o', label='Scratch')
plt.title('kNN (Scratch) Test Accuracy vs. k')
plt.xlabel('k (Number of Neighbors)')
plt.ylabel('Test Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()
